In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import healpy as hp
import scipy
from numba import jit, njit, prange, set_num_threads
from tqdm.notebook import tqdm
from helper_funcs import *

We divide the $\ell$-range $\left[\ell_{\min }, \ell_{\max }\right]$ into subintervals denoted by $\Delta_i=\left[\ell_i, \ell_{i+1}-1\right]$ where $i=0, \ldots,\left(N_{\text {bins }}-1\right)$ and $\ell_{N_{\text {bins }}}=\ell_{\max }+1$, so that the filtered maps are:

\begin{equation*}
M_i^p(\Omega)=\sum_{\ell \in \Delta_i} \sum_{m=-\ell}^{+\ell} a_{\ell m}^p Y_{\ell m}(\hat{\Omega}) \tag{2.5}
\end{equation*}

and we use these instead of $M_{\ell}^p$ in the expression for the bispectrum (2.5). The binned bispectrum is:

\begin{equation*}
B_{i_1 i_2 i_3}^{p_1 p_2 p_3, \mathrm{obs}}=\frac{1}{\Xi_{i_1 i_2 i_3}} \int d \hat{\Omega} M_{i_1}^{p_1, \mathrm{obs}}(\hat{\Omega}) M_{i_2}^{p_2, \mathrm{obs}}(\hat{\Omega}) M_{i_3}^{p_3, \mathrm{obs}}(\hat{\Omega}) \tag{2.6}
\end{equation*}

where $\Xi_{i_1 i_2 i_3}$ is the number of $\ell$ triplets within the $\left(i_1, i_2, i_3\right)$ bin triplet satisfying the triangle inequality and parity condition selection rule. Because of this normalization factor, $B_{i_1 i_2 i_3}^{p_1 p_2 p_3}$ may be considered an average over all valid $B_{\ell_1 \ell_2 \ell_3}^{p_1 p_2 p_3}$ inside the bin triplet.

In [7]:
def compute_filtered_maps(lmin, lmax, num_bins, alms, *omega):
    theta, phi = omega
    # scipy.special.sph_harm(l, m, theta, phi)
    # for m in range(-l, l + 1):
    #     alms[l][m]

In [3]:
@njit(parallel=True)
def count_valid_trips(i1, i2, i3, num_threads=16):
    """
    Counts the number of valid ell-triplet configurations in a single bin.
    Validity is determined as satisfying the parity condition selection
    rule and the triangle inequality.

    Inputs:
    i1 (int) : bin1
    i2 (int) : bin2
    i3 (int) : bin3
    num_threads (int) : number of threads to parallelize on

    Returns:
    (int) : number of valid configurations of ell-triplets.
    """

    set_num_threads(num_threads)

    configs = 0

    for l1 in prange(i1 + 1):
        for l2 in range(i2 + 1):
            for l3 in range(i3 + 1):
                if (l1 + l2 + l3) % 2 == 0 and \
                        np.abs(l1 - l2) <= l3 <= l1 + l2:
                    configs += 1
    
    return configs

In [39]:
# def P(l, m, x):
# 	pmm = 1.0
# 	if m > 0:
# 		somx2 = np.sqrt((1.0 - x) * (1.0 + x))
# 		fact = 1.0
# 		for i in range(1, m + 1):
# 			pmm *= (-fact) * somx2
# 			fact += 2.0
	
# 	if l == m:
# 		return pmm * np.ones(x.shape)
	
# 	pmmp1 = x * (2.0*m+1.0) * pmm
	
# 	if l == m + 1:
# 		return pmmp1
	
# 	pll = np.zeros(x.shape)
# 	for ll in range(m + 2, l + 1):
# 		pll = ((2.0 * ll - 1.0) * x * pmmp1 - (ll + m - 1.0) * pmm) / (ll - m)
# 		pmm = pmmp1
# 		pmmp1 = pll
	
# 	return pll

In [ ]:
# def ylm_norm_factor(l, m):
#     """
#     Finds the normalization factor for the spherical harmonic function
#     a given ell and m.

#     Inputs:
#     l : (int) ell-value
#     m : (int) m-value

#     Returns:
#     (int) : normalization factor for spherical harmonic function
#     """
#     return np.sqrt((2 * l + 1)/(4 * np.pi) \
#         * ((math.factorial(l - m))/(math.factorial(l + m))))

# def ylm(omega, l, m):
#     """
#     The spherical harmonic functions.

#     Inputs:
#     omega : (var) solid-angle omega
#     l : (int) ell-value
#     m : (int) m-value

#     Returns:
#     (func) : spherical harmonic function for a given ell, m input.
#     """